## **Feature Extract**

두 도메인의 feature를 추출하기에 앞서, 아날로그 형태의 음악 데이터를 학습에 사용하기 위해서는 먼저 windowing(=sampling) 과정을 거쳐주어야 합니다. 또 표준 샘플링 레이트는 44100Hz를 주로 사용합니다. 즉, 1초를 44100등분하여 음악값들을 샘플링 해오겠다는 것입니다.

#### **[방법 1] Rhythm domain feature extraction**

여러가지 rhythm 도메인의 feature 중 Tempogram을 추출합니다.

![image](pic/several_data_p3_4.png)

rhythm feature 추출 흐름도는 위와 같습니다.

리듬이란, 일반적으로 음악의 기본이 되는 규칙적인 펄스 또는 음악에서 전반적으로 유지되는 반복적 패턴을 의미합니다. 이 때 일정한 간격으로 발생하는 펄스를 비트라고 하며, 비트의 강약이 변화하여 만들어지는 패턴을 박자라고 칭합니다.

리듬과 그에 대한 박자는 사실 다양한 변수들이 혼합된 개념이기에 정확히 한 문장으로 정의하기는 힘들지만, 보통 음악적 리듬의 박자 구조는 일정한 간격을 지닌 시간 축 격자에 표시할 수 있는 연속적인 리듬 이벤트를 의미합니다.

음악은 박자 시스템에 기반한 리듬 구조를 가지고 있기 때문에 박자 구조를 분석하는 것이 리듬 분석을 위해서는 필수적이라고 볼 수 있습니다. 그리고 이러한 음악의 리듬 패턴을 분석하는데 사용되는 시간적 feature 중 하나가 Tempogram입니다.

그렇다면 음악의 Tempogram은 어떻게 추출할 수 있을까요?

음악의 Tempogram을 추출하기 위해서는 먼저 windowing을 거쳐 시간별로 쪼개진 데이터를 librosa.onset.onset_strength() 함수의 입력으로 주어 onset 지점을 탐지해야 합니다. Onset이란, 주로 박자의 첫 번째 비트나 강조되는 리듬 변화가 발생하는 시점을 의미합니다. 그렇기 때문에 onset 지점을 검출하는 것이 곧 음악의 리듬을 분석하는 것이고, 이를 수행하는 다양한 방식이 있지만 대표적으로 주파수 스펙트럼의 변화, 에너지 변화 등을 파악하여 onset 지점을 검출합니다.

다음으로는 이렇게 탐지된 onset 지점을 작은 프레임 단위로 나누고 각 프레임에 대해 autocorrelation(자기상관) 값을 계산합니다. 이는 각 onset 지점의 자기상관성을 분석한 그래프로, 시간 지연된 타 onset 지점과의 유사도를 계산하여 주기성과 리듬 정보를 파악할 수 있게 됩니다.

![image](pic/several_data_p3_5.png)

위 그림에서 가운데 그래프가 onset 검출 점수 그래프에 해당합니다. 앞선 과정을 거쳐 시간 지연 축에 대한 자기상관함수 값을 얻었다면, 위 값들을 통해 Tempogram 값을 얻어야 합니다. 현재 자기상관함수는 시간 지연 축에 대해 나타나 있기 때문에, 템포에 대한 축으로 변경해주어야 이 값들을 유의미하게 활용할 수 있습니다.
이러한 변형은 아래 두 과정을 거쳐 이루어집니다.

- Tempo (in BPM) = 60 / Timelag (in sec) 식을 적용하여 x축 변환
- 음악적으로 유의미한 tempo 축을 갖도록 보간 적용

![image](pic/several_data_p3_6.png)

우선 첫 번째 식을 적용해 자기상관함수의 time-lag 축을 tempo(BPM)축으로 바꾸어줍니다. 이렇게 되면 수식에 따라 tempo 축의 범위가 선형적이지 않은데, 이를 펴주기 위해 tempo 축에 대한 보간을 수행합니다.

위와 같은 과정을 거쳐 음악의 Tempogram을 얻을 수 있습니다. 과정이 어렵고 생소할 수 있지만 librosa 라이브러리에서는 단순히 앞서 얻은 onset_strength 값을 librosa.feature.tempogram()의 인자로 넘겨줌으로써 음악의 Tempogram을 얻을 수 있습니다.

#### **[방법 2] Spectral domain feature extraction**
spectral 도메인의 여러 feature 중 Chromagram과 MFCC를 추출합니다.

![image](pic/several_data_p3_7.png)

spectral feature 추출 흐름도는 위와 같습니다.

마찬가지로 처음엔 windowing을 거쳐 음성 데이터를 이산화합니다.

Windowing 과정을 거쳐 잘게 나누어진 프레임들에 각각 STFT(Short Time Fourier Transform)를 적용해 주파수 도메인으로 변환된 spectrogram을 얻는 과정을 거칩니다. 이에 대해서는 아래에서 계속 설명드리겠습니다.

![image](pic/several_data_p3_8.png)

STFT(librosa.stft())는 음성 신호의 시간적 정보를 잘 활용하기 위해 각 프레임마다 FFT(Fast Fourier Transform)을 적용하여 시간 도메인의 데이터를 주파수 도메인으로 변환해주는 과정입니다. 일정 시간 영역에는 수많은 주파수가 혼합되어 있어 정보를 분석하기 어렵기 때문에 음성의 고유 배음구조를 유추할 수 있도록 신호를 위 그림과 같이 주파수 영역으로 변환시켜주는 것입니다.

![image](pic/several_data_p3_9.png)

FFT를 음성의 전체 시간 영역에 한 번에 적용하게 되면 음성의 시계열적 특성 정보를 제대로 활용할 수 없게 됩니다. 따라서 전체 시간 영역에 대해 FFT를 한 번에 적용하는 것이 아닌, 위 그림과 같이 잘게 나뉜 영역에 대해 여러 번(총 window 개수만큼) FFT를 적용하여 주파수 도메인으로 변환(=STFT)해준다면 주파수 정보 뿐만 아니라 시간적 순서와 관련된 time frame에 대한 정보까지 얻을 수 있게 됩니다.

STFT를 적용하여 얻은 값(=spectrum)의 magnitude를 사용하기 위해 절대값을 취하여 spectrogram을 얻고, 이렇게 얻은 spectrogram을 제곱하여 power spectrogram을 얻습니다. 다음 함수의 입력으로 spectrogram이 아니라 power spectrogram이 사용되는 이유는 주파수 도메인에서 시간에 따른 신호의 에너지를 강조해줄 수 있기 때문입니다. 제곱을 함으로써 음악 신호의 패턴을 더 쉽게 감지하고, 음악에서 특징점이 될 수 있는 부분을 잡아내는 데에 더욱 도움되는 값을 얻어낼 수 있는 것입니다.

##### **1) Chromagram**

Chromagram이란 음악의 주파수 정보를 사용해 음악의 음계 기반 특성을 파악하는 방법입니다. 음악의 주파수 영역에서 각 음높이에 해당하는 성분들의 빈도 정보를 표현하여 음악의 화음과 멜로디 정보를 추출할 수 있게 되는 것입니다.

Chromagram은 기본값으로 하나의 프레임을 12차원의 벡터로 표현하는데, 여기서 12란 숫자는 서양 음악에서 사용되는 12음계 체계(도, 도#, 레, 레#, 미, 파, 파#, 솔, 솔#, 라, 라#, 시)를 기반으로 설정된 것입니다. 참고로 이 12음계를 chromatic scale이라 부르고, 이 음계를 기반으로 분석하는 것이 Chromagram인 것입니다.

![image](pic/several_data_p3_10.png)

앞서 얻은 power spectrogram을 입력으로 주어 각 프레임에서 12개의 음들이 얼마나 등장하는지를 나타내는 크로마 밴드를 구하고, 이에 해당하는 파형 크기의 제곱 값을 합하여 Chromagram을 만들어냅니다. 위 그림은 12개의 음계를 바탕으로 만들어낸 음악의 Chromagram입니다.

Chromagram은 librosa.feature.chroma_stft()에 power spectrogram을 입력으로 주어 추출할 수 있습니다.

##### **2) MFCC**

MFCC는 Mel Frequency Cepstral Coefficient의 약자입니다. 다양한 음성 feature가 존재하지만, MFCC는 사람의 청각 기관 특성을 가장 잘 살려 설계된 feature 중 하나에 해당하고 현재까지도 음성, 음악 관련 task를 수행하기 위해 많이 사용됩니다.

다시 전체 파이프라인 그림으로 올라가보시면 MFCC는 power spectrogram에 melfilter 필터를 적용해 melspectrogram(librosa.feature.melspectrogram())을 추출하고, 켑스트럴 분석(DCT, Discrete Cosine Transform, librosa.power_to_db())를 거쳐 얻게됩니다.

**Melspectrogram**<br>

실제로 사람의 달팽이관은 저주파 대역을 감지하는 부분은 굵고, 고주파 대역을 감지하는 부분은 얇습니다. Melfilter는 이렇게 사람의 청각기관이 높은 주파수보다 낮은 주파수 대역에 더욱 민감하게 반응한다는 사실을 반영하여 power spectrogram을 조정하는 역할을 수행합니다.

![image](pic/several_data_p3_11.png)

위 그림은 melfilter인데 상대적으로 높은 주파수 대역으로 갈수록 필터가 감소하는 것을 볼 수 있습니다. 이를 적용해 사람이 더욱 민감하게 반응하는 주파수 대역을 강조한 값으로 변환된 melspectrogram을 얻을 수 있게 됩니다.

**Cepstral analysis**<br>

다음은 앞서 얻은 melspectrogram에 켑스트럴 분석 과정을 적용하고 이를 통해 최종적인 MFCC feature를 얻을 수 있습니다.
과정상으로 보면, melspectrogram을 log scale로 만들어 log spectrogram을 얻고 이에 DCT(Discrete Cosine Transform)를 적용하여 최종 MFCC feature를 추출하게 되는 것입니다.

여기서 log scale로 만드는 이유는, 사람의 소리 인식이 log scale에 가깝기 때문입니다. 다시 말해 사람이 두 배 큰 소리라고 인식하기 위해서는, 실제로 에너지가 100배 큰 소리여야 한다는 특성을 feature 추출 과정에 적용해주었다는 뜻입니다. 다음으로 DCT는 대부분의 실제 신호가 우함수이기 때문에 입력신호를 우함수인 코사인 함수의 합으로 표현해주는 과정입니다. DCT의 변환 결과는 계수 형태로 나타나며, log spectrogram에 DCT를 적용했을 때 주파수 성분으로 분해하여 얻은 계수 중 주로 사용되는 일부 계수들만 선택하면 그것이 MFCC feature로서 사용되는 것입니다.

In [1]:
# 라이브러리 임포트
import math
import os
import random

import numpy as np
import pandas as pd

import librosa
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from tqdm import tqdm

In [2]:
# 하이퍼파라미터
args = {
    "train_path" : "/kaggle/input/2024-outta-basic-p-3/train/train",
    "test_path" : "/kaggle/input/2024-outta-basic-p-3/test/test",
    "submit_path" : "/kaggle/input/2024-outta-basic-p-3/sample_submission.csv",
    "extract_features" : "spectral",  # "rhythm"과 "spectral" 중에 선택하세요.
    "batch_size" : 32,
    "num_labels" : 2,
    "epochs" : 40,
    "lr" : 2e-5, 
    "eps" : 1e-8,
    "seed_val" : 42     # 절대 수정하지 마세요.
}

In [3]:
# 랜덤시드 고정하기
seed = args["seed_val"]
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available() : 
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# 디바이스 선택
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
%pip install torchsummary
from torchsummary import summary as summary_## 모델 정보를 확인하기 위해 torchsummary 함수 import

## 모델의 형태를 출력하기 위한 함수 
def summary_model(model, input_shape=(384,)):
    model = model.to(device)
    summary_(model, input_shape) ## (model, (input shape))

Note: you may need to restart the kernel to use updated packages.


In [5]:
# (선택) Ipython 라이브러리를 이용해 학습에 사용될 음성을 직접 들어볼 수 있습니다.
from IPython.display import Audio, display

filename = ''  # 파일 주소
y, sr = librosa.load(filename)
audio_wdt = Audio(data=y,rate=sr)
display(audio_wdt)

/tmp/ipykernel_33/1112050501.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(filename)
/opt/conda/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: ''

# **1. 데이터**

## **(1) Label Map**
{'강아지' : 0, '고양이' : 1} 등과 같은 형식으로, 머신러닝, 딥러닝 모델들은 feature나 label의 값들이 숫자(정수/실수)인 것만 처리할 수 있기 때문에, 문자열일 경우 숫자형으로 변환하여 처리합니다.

In [6]:
label_map = {'blues': 0, 
             'classical': 1, 
             'country': 2, 
             'disco': 3, 
             'hiphop': 4, 
             'jazz': 5, 
             'metal': 6, 
             'pop': 7, 
             'reggae': 8, 
             'rock': 9}

## **(2) Feature Extract**
컴퓨터에 입력할 수 있도록 샘플링과 양자화를 거쳤다고 해도, 이를 바로 분류기에 넣는 것은 높은 성능을 보장할 수 없습니다.<br>
이는 음성 정보 안에 여러 주파수가 섞여 있으며 방대한 정보를 담고 있기 때문입니다.<br>
따라서 데이터 중 음성의 대표적인 성질을 나타낼 수 있는 handcrafted feature를 추출하여 사용하는 것이 필수적입니다.<br>
데이터에서 어떠한 특징을 어떠한 방식으로 추출하는지에 따라 분류기 성능에 큰 영향을 끼칠 수 있기 때문에 feature 추출을 위해서는 신중한 설계가 선행되어야 합니다.<br>

- 참고 1 : https://librosa.org/doc/latest/index.html
- 참고 2 : https://wikidocs.net/192879

#### **Method 1. Rhythm**
- Method 1에서는 rhythm 도메인의 두 가지 feature인 autocorrelation tempogram과 fourier tempogram을 추출합니다.
- 단, librosa.load() 함수를 통해 얻은 time-series (y, sr)를 입력으로 주어 두 feature를 얻는 것을 목표로 합니다.
- 각 단계별 세부 내용은 아래 함수를 참고하시기 바랍니다.

(1-1) librosa.load(): extract_rhythm_features() 함수의 인자로 넘겨 받은 file_path에 대하여 1초 당 22050개의 샘플을 추출한 time-series를 load합니다.<br>
(1-2) librosa.onset.onset_strength(): 앞서 load한 time-series를 함수의 입력으로 주어 onset_envelope를 추출합니다.<br>
(1-3) librosa.feature.tempogram(): 앞서 추출한 onset_envelope와 sr을 함수의 입력으로 주어 autocorrelation tempogram feature를 추출합니다.<br>
(1-4) autocorrelation tempogram에 대해 시간 축으로 평균을 내고 절대값을 취해 복소수를 제거함으로써 tempogram_feature를 얻습니다.

In [7]:
def extract_rhythm_features(file_path):
    # 반환할 feature list 선언
    feature = []
    
    # (1-1) librosa.load(): extract_rhythm_features() 함수의 인자로 넘겨 받은 file_path에 대하여 1초 당 22050개의 샘플을 추출한 time-series를 load합니다.
    y, sr = librosa.load(file_path, sr=22050)
    
    # (1-2) librosa.onset.onset_strength(): 앞서 load한 time-series를 함수의 입력으로 주어 onset_envelope를 추출합니다.
    onset_envelope = librosa.onset.onset_strength(y=y, sr=sr)
    
    # autocorrelation tempogram
    # (1-3) librosa.feature.tempogram(): 앞서 추출한 onset_envelope와 sr을 함수의 입력으로 주어 autocorrelation tempogram feature를 추출합니다. 함수의 입력 변수를 잘 확인하세요.
    tempogram = librosa.feature.tempogram(onset_envelope=onset_envelope, sr=sr)
    
    # (1-4) autocorrelation tempogram에 대해 시간 축으로 평균을 내고 절대값을 취해 복소수를 제거함으로써 tempogram_feature를 얻습니다.
    tempogram_feature = np.mean(np.abs(tempogram), axis=1)   # (384, 1293) -> (384, ), 복소수를 없애기 위해 절대값 처리
    
    feature = tempogram_feature
    
    return feature

#### **Method 2. Spectral**
- Method 2에서는 spectral 도메인의 두 가지 feature인 MFCC와 chromagram을 추출합니다.
- 단, librosa 라이브러리의 함수를 통해 spectrogram, power_spectrogram, melspectrogram, melspectrogram_db를 순차적으로 얻고, 이를 입력으로 주어 두 feature를 얻는 것을 목표로 합니다.
- 각 단계별 세부 내용은 아래 함수를 참고하시기 바랍니다.

(2-1) librosa.load(): extract_spectral_features() 함수의 인자로 넘겨 받은 file_path에 대하여 1초 당 22050개의 샘플을 추출한 time-series를 load합니다.<br>
(2-2) librosa.stft(): 앞서 load한 time-series를 함수의 입력으로 주어 stft를 추출하고, 절대값을 취해 spectrogram을 추출합니다.<br>
(2-3) 앞서 얻은 spectrogram에 제곱 연산을 취해 power_spectrogram을 얻습니다.<br>
(2-4) librosa.feature.melspectrogram(): 앞서 얻은 power_spectrogram을 함수의 입력으로 주어 melspectrogram을 추출합니다.<br>
(2-5) librosa.power_to_db(): 앞서 얻은 melspectrogram을 함수의 입력으로 주어 db scale로 변환된 melspectrogram_db를 추출합니다.<br>
(2-6) librosa.feature.chroma_stft(): (2-3)에서 얻은 power_spectrogram을 함수의 입력으로 주어 chromagram을 추출합니다.<br>
(2-7) chromagram에 대해 시간 축으로 평균을 내어 feature로 사용할 수 있도록 변환합니다.<br>
(2-8) librosa.feature.mfcc(): (2-5)에서 얻은 melspectrogram_db를 함수의 입력으로 주어 mfcc를 추출합니다.<br>
(2-9) mfcc에 대해 시간 축으로 평균을 내어 feature로 사용할 수 있도록 변환합니다.<br>
(2-10) chromagram과 mfcc를 하나의 feature로 반환할 수 있도록 concatenate를 수행합니다.

In [8]:
def extract_spectral_features(file_path):
    # 반환할 feature list 선언
    feature = []
    
    # (2-1) librosa.load(): extract_spectral_features() 함수의 인자로 넘겨 받은 file_path에 대하여 1초 당 22050개의 샘플을 추출한 time-series를 load합니다.  
    y, sr = librosa.load(file_path, sr=22050)

    # (2-2) librosa.stft(): 앞서 load한 time-series를 함수의 입력으로 주어 stft를 추출하고, 절대값을 취해 복소수를 제거한 spectrogram을 추출합니다.  
    # * 이 때, 93ms의 물리적 간격으로 나뉘어 spectrogram이 생성될 수 있도록 하이퍼파라미터 n_fft를 조절합니다. 어떠한 값을 넣어야 하는지는 공식 문서를 참고하세요.
    n_fft = 2048
    n_fft = librosa.stft(y, n_fft=n_fft)
    spectrogram = np.abs(n_fft)
    
    # (2-3) 앞서 얻은 spectrogram에 제곱 연산을 취해 power_spectrogram을 얻습니다.  
    power_spectrogram = spectrogram ** 2
    power_spectrogram = power_spectrogram.astype(np.float32)
    # (2-4) librosa.feature.melspectrogram(): 앞서 얻은 power_spectrogram을 함수의 입력으로 주어 melspectrogram을 추출합니다.  
    melspectrogram = librosa.feature.melspectrogram(S=power_spectrogram, sr=sr, n_fft=n_fft)
    
    # (2-5) librosa.power_to_db(): 앞서 얻은 melspectrogram을 함수의 입력으로 주어 db scale로 변환된 melspectrogram_db를 추출합니다.  
    melspectrogram_db = librosa.power_to_db(melspectrogram)
    
    # chromagram
    # (2-6) librosa.feature.chroma_stft(): (2-3)에서 얻은 power_spectrogram을 함수의 입력으로 주어 chromagram을 추출합니다. 함수의 입력 변수를 잘 확인하세요.
    chromagram = librosa.feature.chroma_stft(S=power_spectrogram, sr=sr)  # (12, 1293)
    
    # (2-7) chromagram에 대해 시간 축으로 평균을 내어 chromagram_feature를 얻습니다. 
    chromagram_feature = np.mean(chromagram, axis=1)  # (12, )
    
    # mfcc
    # (2-8) librosa.feature.mfcc(): (2-5)에서 얻은 melspectrogram_db를 함수의 입력으로 주어 mfcc를 추출합니다. 함수의 입력 변수를 잘 확인하세요.
    mfcc = librosa.feature.mfcc(S=melspectrogram_db, sr=sr, n_mfcc=20)  # (20, 1293)
    
    # (2-9) mfcc에 대해 시간 축으로 평균을 내어 mfcc_feature를 얻습니다.  
    mfcc_feature = np.mean(mfcc, axis=1)  # (20, )
    
    # (2-10) chromagram과 mfcc를 하나의 feature로 반환할 수 있도록 concatenate를 수행합니다.
    feature = np.concatenate([chromagram_feature, mfcc_feature])

    return feature

In [9]:
import numpy as np
import librosa

def extract_spectral_features(file_path):
    # 반환할 feature list 선언
    feature = []
    
    # (2-1) librosa.load(): extract_spectral_features() 함수의 인자로 넘겨 받은 file_path에 대하여 1초 당 22050개의 샘플을 추출한 time-series를 load합니다.  
    y, sr = librosa.load(file_path, sr=22050)

    # (2-2) librosa.stft(): 앞서 load한 time-series를 함수의 입력으로 주어 stft를 추출하고, 절대값을 취해 복소수를 제거한 spectrogram을 추출합니다.  
    n_fft = 2048
    stft_result = librosa.stft(y, n_fft=n_fft)  # stft 결과를 별도의 변수에 저장
    spectrogram = np.abs(stft_result)
    
    # (2-3) 앞서 얻은 spectrogram에 제곱 연산을 취해 power_spectrogram을 얻습니다.  
    power_spectrogram = spectrogram ** 2
    power_spectrogram = power_spectrogram.astype(np.float32)
    
    # (2-4) librosa.feature.melspectrogram(): 앞서 얻은 power_spectrogram을 함수의 입력으로 주어 melspectrogram을 추출합니다.  
    melspectrogram = librosa.feature.melspectrogram(S=power_spectrogram, sr=sr, n_fft=n_fft)
    
    # (2-5) librosa.power_to_db(): 앞서 얻은 melspectrogram을 함수의 입력으로 주어 db scale로 변환된 melspectrogram_db를 추출합니다.  
    melspectrogram_db = librosa.power_to_db(melspectrogram)
    
    # chromagram
    # (2-6) librosa.feature.chroma_stft(): (2-3)에서 얻은 power_spectrogram을 함수의 입력으로 주어 chromagram을 추출합니다.
    chromagram = librosa.feature.chroma_stft(S=power_spectrogram, sr=sr)
    
    # (2-7) chromagram에 대해 시간 축으로 평균을 내어 chromagram_feature를 얻습니다. 
    chromagram_feature = np.mean(chromagram, axis=1)
    
    # mfcc
    # (2-8) librosa.feature.mfcc(): (2-5)에서 얻은 melspectrogram_db를 함수의 입력으로 주어 mfcc를 추출합니다.
    mfcc = librosa.feature.mfcc(S=melspectrogram_db, sr=sr, n_mfcc=20)
    
    # (2-9) mfcc에 대해 시간 축으로 평균을 내어 mfcc_feature를 얻습니다.  
    mfcc_feature = np.mean(mfcc, axis=1)
    
    # (2-10) chromagram과 mfcc를 하나의 feature로 반환할 수 있도록 concatenate를 수행합니다.
    feature = np.concatenate([chromagram_feature, mfcc_feature])

    return feature


## **(3) Custom Dataset**
Custom Dataset 클래스는 크게 **__init__(), __len__(), 그리고 __getitem__()** 3개의 함수로 구현해야 합니다. 

1.  __init__()
    - Dataset instance를 생성할 때 한번만 실행되는 함수로, 입력 이미지의 디렉토리와 라벨 정보 그리고 transform을 초기화 합니다. 

2. __len__()
    - 데이터셋의 샘플 개수를 반환하는 함수 입니다. 
    
3. __getitem__()
    - 주어진 인덱스에 해당하는 데이터 샘플을 데이터셋에서 불러오고 반환하는 함수 입니다.

In [10]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root_path, split, extract_features, label_map):
        self.split = split.upper()
        self.root_path = root_path
        self.extract_features = extract_features
        self.label_map = label_map
        
        self.music = []
        self.label = []
        
        # (3-1) test 데이터 디렉토리 경로 초기화
        if self.split == 'TEST':
            musics = sorted(os.listdir(self.root_path))
            for music_ in musics:
              music_path = os.path.join(self.root_path, music_)
              if os.path.isfile(music_path):
                  self.music.append(music_path)
            
        # (3-2) train 데이터 디렉토리 경로, 라벨 초기화
        else:
            genres = sorted(os.listdir(self.root_path))
            for genre_ in genres:
                genre_path = os.path.join(self.root_path, genre_)
                if not os.path.isdir(genre_path):
                    continue
                musics = sorted(os.listdir(genre_path))
                for music_ in musics:
                  music_path = os.path.join(genre_path, music_)
                  if os.path.isfile(music_path):
                      self.music.append(music_path)
                      self.label.append(self.label_map[genre_])
                    
            
    # 전체 데이터 샘플 개수 반환
    def __len__(self):
        return len(self.music)
    
    # 주어진 인덱스에 해당하는 데이터 반환
    def __getitem__(self, idx):        
        # (3-3) 음악 데이터에 feature extract 적용
        if self.extract_features == 'spectral':
            music_feature = extract_spectral_features(self.music[idx])
        elif self.extract_features == 'rhythm':
            music_feature = extract_rhythm_features(self.music[idx])
        
        # (3-4) test에 사용할 데이터 반환
        if self.split == 'TEST':
            return music_feature
        # (3-5) train에 사용할 데이터 반환
        else:
            label = self.label[idx]
            return music_feature, label

### **(4) 데이터셋과 데이터로더**

In [11]:
# (4-1) 훈련 및 테스트 데이터셋 로드
train_dataset = CustomDataset(root_path=args['train_path'], split='TRAIN', extract_features=args["extract_features"],label_map=label_map)
test_dataset = CustomDataset(root_path=args['test_path'], split='TEST', extract_features=args["extract_features"], label_map=label_map) 

# (4-2) 데이터로더 정의
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [12]:
print(f'Train Music Data\'s shape : {train_dataset.__getitem__(0)[0].shape}')   # train_dataset의 반환값 : music_feature, label
print(f'Test Music Data\'s shape : {test_dataset.__getitem__(0).shape}')        # test_dataset의 반환값 : music_feature

Train Music Data's shape : (32,)
Test Music Data's shape : (32,)


# **3. 모델**
모델을 직접 설계하여 프로젝트를 수행하세요.

직접 쌓기만 하신다면, 어떤 모델이든 사용 가능합니다.

아래는 베이스라인 모델입니다.

> ```
>         Layer (type)               Output Shape         Param #
> =================================================================
>             Conv1d-1              [-1, 64, 384]             256
>               ReLU-2              [-1, 64, 384]               0
>             Conv1d-3              [-1, 64, 384]          12,352
>               ReLU-4              [-1, 64, 384]               0
>          MaxPool1d-5              [-1, 64, 192]               0
>             Conv1d-6             [-1, 128, 192]          24,704
>               ReLU-7             [-1, 128, 192]               0
>             Conv1d-8             [-1, 128, 192]          49,280
>               ReLU-9             [-1, 128, 192]               0
>         MaxPool1d-10              [-1, 128, 96]               0
>            Conv1d-11              [-1, 256, 96]          98,560
>              ReLU-12              [-1, 256, 96]               0
>            Conv1d-13              [-1, 256, 96]         196,864
>              ReLU-14              [-1, 256, 96]               0
>            Conv1d-15              [-1, 256, 96]         196,864
>              ReLU-16              [-1, 256, 96]               0
>         MaxPool1d-17              [-1, 256, 48]               0
>            Conv1d-18              [-1, 512, 48]         393,728
>              ReLU-19              [-1, 512, 48]               0
>            Conv1d-20              [-1, 512, 48]         786,944
>              ReLU-21              [-1, 512, 48]               0
>            Conv1d-22              [-1, 512, 48]         786,944
>              ReLU-23              [-1, 512, 48]               0
>         MaxPool1d-24              [-1, 512, 24]               0
>            Conv1d-25              [-1, 512, 24]         786,944
>              ReLU-26              [-1, 512, 24]               0
>            Conv1d-27              [-1, 512, 24]         786,944
>              ReLU-28              [-1, 512, 24]               0
>            Conv1d-29              [-1, 512, 24]         786,944
>              ReLU-30              [-1, 512, 24]               0
>         MaxPool1d-31              [-1, 512, 12]               0
> AdaptiveAvgPool1d-32               [-1, 512, 1]               0
>           Flatten-33                  [-1, 512]               0
>            Linear-34                   [-1, 10]           5,130
> =================================================================
> Total params: 4,912,458
> Trainable params: 4,912,458
> Non-trainable params: 0
> ----------------------------------------------------------------
> Input size (MB): 0.00
> Forward/backward pass size (MB): 4.74
> Params size (MB): 18.74
> Estimated Total Size (MB): 23.48
> ----------------------------------------------------------------
> ```


In [13]:
# (5) 모델 설계
class Model(nn.Module):
    def __init__(self):
        """
        모델 초기화 함수입니다.
        """
        super(Model, self).__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2))
            
        
        self.conv_block2 = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2))
            
        
        self.conv_block3 = nn.Sequential(
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Conv1d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2))
        
        self.conv_block4 = nn.Sequential(
            nn.Conv1d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Conv1d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2))
        
        self.conv_block5 = nn.Sequential(
            nn.Conv1d(in_channels=512, out_channels=1024, kernel_size=3, padding=1),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Conv1d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1),
            nn.AdaptiveMaxPool1d(output_size=1))  # AdaptiveMaxPool1d로 수정
    
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(in_features=1024, out_features=10))

    def forward(self, x):
        """
        모델의 순전파 함수입니다.

        Args:
            x (torch.Tensor): 입력 데이터

        Returns:
            torch.Tensor: 출력 예측값
        """
        x = x.unsqueeze(-1)
        x = x.permute(0, 2, 1)
        
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.conv_block5(x)
        
        x = self.fc(x)
        return x


In [14]:
# 모델을 장치로 이동
model = Model().to(device)

# 모델 요약 정보 출력
summary_model(model, input_shape=train_dataset.__getitem__(0)[0].shape)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 64, 32]             256
       BatchNorm1d-2               [-1, 64, 32]             128
              ReLU-3               [-1, 64, 32]               0
            Conv1d-4               [-1, 64, 32]          12,352
              ReLU-5               [-1, 64, 32]               0
            Conv1d-6               [-1, 64, 32]          12,352
              ReLU-7               [-1, 64, 32]               0
            Conv1d-8               [-1, 64, 32]          12,352
              ReLU-9               [-1, 64, 32]               0
        MaxPool1d-10               [-1, 64, 16]               0
           Conv1d-11              [-1, 128, 16]          24,704
      BatchNorm1d-12              [-1, 128, 16]             256
             ReLU-13              [-1, 128, 16]               0
           Conv1d-14              [-1, 

# **4. 학습**

In [15]:
def train(train_dataloader, model, device, args):
    """
    모델을 훈련시키는 함수입니다.

    Args:
        train_dataloader (DataLoader): 훈련 데이터가 포함된 데이터로더
        model (nn.Module): 훈련할 신경망 모델
        device (torch.device): 모델과 데이터를 올릴 장치 (CPU 또는 GPU)
        args (dict): 훈련 설정을 포함한 딕셔너리 (예: 에포크 수)

    Returns:
        None
    """
    # (6-1) 옵티마이저와 손실 함수 정의
    optimizer = torch.optim.Adam(model.parameters(), lr=args["lr"], eps=args["eps"])
    loss_fn = nn.CrossEntropyLoss().to(device)

    # 모델의 모든 파라미터의 기울기를 0으로 초기화
    model.zero_grad()

    # 지정된 에포크 수만큼 훈련 반복
    for epoch in range(args["epochs"]):
        print(f'Epoch {epoch + 1}/{args["epochs"]}')
        
        train_loss = 0
        train_acc = 0
        
        # 훈련 데이터로더에서 데이터를 반복적으로 가져옴
        for data, label in tqdm(train_dataloader):
            # 데이터를 부동 소수점 형식으로 변환하고 장치에 올림
            data = data.float().to(device)
            label = label.to(device)
            
            # (6-2) 모델을 사용하여 예측 수행
            pred = model(data)
            
            # (6-3) 예측값과 실제 라벨을 사용하여 손실 계산
            loss = loss_fn(pred, label)
            
            # (6-4) 옵티마이저의 기울기를 초기화
            optimizer.zero_grad()
            
            # (6-5) 역전파를 통해 기울기 계산
            loss.backward()
            
            # (6-6) 옵티마이저를 사용하여 모델의 파라미터 업데이트
            optimizer.step()
            
            # 총 손실 계산
            train_loss += loss.item()
            
            # 예측값에서 가장 높은 값의 인덱스를 선택
            pred = pred.argmax(dim=1)
            # 정확도 계산
            train_acc += (pred == label).sum().item()
        
        # 평균 손실과 정확도 계산
        train_loss /= len(train_dataloader)
        train_acc /= len(train_dataloader.dataset)
            
        # 모델과 옵티마이저 상태를 저장
        os.makedirs("results", exist_ok=True)
        torch.save({
            'epoch': epoch,
            'model': model,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss.item,
            }, f'./results/model_state_dict_epoch_{epoch+1}.pth')
            
        # 현재 에포크의 손실과 정확도 출력
        print(f'CheckPoint : model_state_dict_epoch_{epoch+1}.pth')
        print(f'Train Loss : {train_loss}, Train Accuracy : {train_acc}\n')
        

if __name__ == "__main__":
    # 훈련 함수 호출
    train(train_dataloader, model, device, args)

Epoch 1/40


100%|██████████| 29/29 [01:51<00:00,  3.83s/it]


CheckPoint : model_state_dict_epoch_1.pth
Train Loss : 2.1953949517217177, Train Accuracy : 0.20568927789934355

Epoch 2/40


100%|██████████| 29/29 [01:40<00:00,  3.47s/it]


CheckPoint : model_state_dict_epoch_2.pth
Train Loss : 2.0025071234538636, Train Accuracy : 0.3479212253829322

Epoch 3/40


...

CheckPoint : model_state_dict_epoch_38.pth
Train Loss : 0.12522499465608392, Train Accuracy : 0.962800875273523

Epoch 39/40


100%|██████████| 29/29 [01:46<00:00,  3.68s/it]


CheckPoint : model_state_dict_epoch_39.pth
Train Loss : 0.13901149468689128, Train Accuracy : 0.9584245076586433

Epoch 40/40


100%|██████████| 29/29 [01:45<00:00,  3.65s/it]


CheckPoint : model_state_dict_epoch_40.pth
Train Loss : 0.12245290928745065, Train Accuracy : 0.9638949671772429



# **5. 평가**

In [16]:
def test(test_dataloader, model, device):
    """
    모델의 예측을 수행하는 함수입니다.

    Args:
        test_dataloader (DataLoader): 테스트 데이터가 포함된 데이터로더
        model (nn.Module): 예측에 사용할 신경망 모델
        device (torch.device): 모델과 데이터를 올릴 장치 (CPU 또는 GPU)

    Returns:
        preds (list): 예측된 클래스의 정수 인덱스 리스트
    """
    # 모델을 평가 모드로 전환
    model.eval()
    # 예측 결과를 저장할 리스트 초기화
    preds = []
    
    # 데이터로더에서 데이터를 반복적으로 가져옴
    for data in tqdm(test_dataloader):
        # 데이터를 부동 소수점 형식으로 변환하고 장치에 올림
        data = data.float().to(device)
        
        # 기울기 계산을 비활성화하여 예측 수행
        with torch.no_grad():
            # (7-1) 모델을 사용하여 예측 수행
            pred = model(data)
            
            # (7-2) argmax를 이용하여 예측 결과에서 가장 높은 값의 인덱스를 선택
            pred = pred.argmax(dim=1).cpu().numpy()
            
            # (7-3) 예측 결과를 리스트에 추가
            preds.extend(pred)
            
    return preds
            

def int_to_label(label_map, predictions_int):
    """
    정수 형태의 예측값 리스트를 클래스 라벨로 변환하는 함수입니다.

    Args:
        label_map (dict): 클래스 라벨과 정수 인덱스 매핑이 저장된 딕셔너리
        predictions_int (list): 정수 형태의 예측값 리스트

    Returns:
        predicted_labels (list): 문자열 형태의 예측 클래스 라벨 리스트
    """
    # 정수 예측값을 클래스 라벨로 변환하여 리스트에 저장
    predicted_labels = [list(label_map.keys())[list(label_map.values()).index(pred)] for pred in predictions_int]
    
    return predicted_labels


if __name__ == "__main__":
    # 예측값을 얻기 위해 test 함수 호출
    preds = test(test_dataloader, model, device)
    
    # 정수 예측값을 클래스 라벨로 변환
    preds = int_to_label(label_map, preds)


100%|██████████| 10/10 [00:32<00:00,  3.29s/it]


In [17]:
submit = pd.read_csv(args["submit_path"])
submit['genre'] = preds
submit.to_csv('submission_p3.csv', index=False)

In [18]:
submit

,music,genre
0,0001.wav,reggae
1,0002.wav,disco
2,0003.wav,jazz
3,0004.wav,rock
4,0005.wav,rock
...,...,...
295,0296.wav,classical
296,0297.wav,metal
297,0298.wav,disco
298,0299.wav,rock
